In [1]:
from pathlib import Path
from itertools import permutations
from nltk.corpus import words
from os.path import exists
import os
import pandas as pd
import re
import csv
import numpy as np
import matplotlib.pyplot as plt
import math
import itertools
import time
import random
import PySimpleGUI as sg
import wordninja

def saveFoundKeywordsAsFile(w_list, keyword, tag):
    fname_base = keyword + '-' + tag
    fpath = Path.cwd() / 'decoded' / fname_base
    np.save(fpath, np.array(w_list))
    print('Found Keywords saved: ', fpath, sep = '')
    return

def saveFoundKeywordsAsTextFile(w_list, keyword, tag):
    fname_base = keyword + '-' + tag + '.txt'
    fpath = Path.cwd() / 'decoded' / fname_base
    with open(fpath, 'w+', newline ='') as text_file:
        csv_writer = csv.writer(text_file)
        csv_writer.writerows(w_list)
        text_file.close();
    return

def rotateRotor(wiring, steps=1, forward=True):
    #print(wiring)
    A2Z = "ABCDEFGHIJKLMNOPQRSTUVWXYZ"
    alen = len(A2Z)
    rotated = ""
    for i in range(len(A2Z)):
        if forward:
            j = (i + steps) % alen
            index = ((ord(wiring[j]) - ord('A') - steps) + alen) % alen
        else:
            j = ((i - steps) + alen) % alen
            index = ((ord(wiring[j]) - ord('A') + steps) + alen) % alen
        rotated = rotated + A2Z[index]
    #print(rotated)
    return rotated

def rotatedRotorList(wiring, steps=1, forward=True):
    w_array = []
    for i in range(len(wiring)):
        rr = rotateRotor(wiring, i)
        w_array.append(rr)
    return w_array

def saveRotatedRotorList(r_list, keyword):
    fname_base = 'rotated_' + keyword + '.txt'
    fpath = Path.cwd() / fname_base
    with open(fpath, 'w+', newline ='') as text_file:
        csv_writer = csv.writer(text_file)
        csv_writer.writerows(r_list)
        text_file.close();
    return

def reversedRotorWiring(wiring):
    revs_list = [" " for i in range(len(wiring))]
    for i in range(len(wiring)):
        c = wiring[i]
        index = ord(c) - ord('A')
        revs_list[index] = chr(ord('A') + i)
    revs = "".join(revs_list)
    return revs

def rotateReversedRotor(wiring, steps=1, forward=True):
    wiring = reversedRotorWiring(wiring)
    A2Z = "ABCDEFGHIJKLMNOPQRSTUVWXYZ"
    alen = len(A2Z)
    rotated = ""
    for i in range(len(A2Z)):
        if forward:
            j = (i + steps) % alen
            index = ((ord(wiring[j]) - ord('A') - steps) + alen) % alen
        else:
            j = ((i - steps) + alen) % alen
            index = ((ord(wiring[j]) - ord('A') + steps) + alen) % alen
        rotated = rotated + A2Z[index]
    #print(rotated)
    return rotated

def rotatedReversedRotorList(wiring, steps=1, forward=True):
    w_array = []
    for i in range(len(wiring)):
        rr = rotateReversedRotor(wiring, i)
        w_array.append(rr)
    return w_array

def saveReversedRotatedRotorList(r_list, keyword):
    fname_base = 'reversed_rotated_' + keyword + '.txt'
    fpath = Path.cwd() / fname_base
    with open(fpath, 'w+', newline ='') as text_file:
        csv_writer = csv.writer(text_file)
        csv_writer.writerows(r_list)
        text_file.close();
    return

def isFreeConsonantMoreThan(word, minlen=0):
    if minlen <= 0:
        return False
    consonants = set("BCDFGHJKLMNPQRSTVWXZ")
    count = 0
    for c in word:
        if c in consonants:
            count = count + 1
            if count > minlen:
                return True
        else:
            count = 0
    return False

def saveSortedListAsTextFile(w_list, keyword, tag):
    fname_base = keyword + '_' + tag + '.txt'
    fpath = Path.cwd() / 'decoded' / keyword
    if not os.path.exists(fpath):
        os.makedirs(fpath)
    fpath_c = fpath / 'checked'
    if not os.path.exists(fpath_c):
        os.makedirs(fpath_c)
    fname = fpath / fname_base
    with open(fname, 'w+', newline ='') as text_file:
        text_file.write("\n".join(str(item) for item in w_list))
        text_file.close();
    return fname

def getListof3Letters(cipher):
    r3list = []
    size = len(cipher) - 2
    if size > 0:
        for i in range(size):
            s = cipher[i:i+3:1]
            r3list.append(s)
    return r3list

def isMatchFound(text, keylist, andAll=False):
    ret = False
    if len(text) > 0 and len(keylist) > 0:
        ret = andAll
        for i in range(len(keylist)):
            index = text.find(keylist[i])
            if not andAll:
                ret = ret or (index >= 0)
            else:
                ret = ret and (index >= 0)
    return ret


In [2]:
class Reflector:
    """Represents a reflector."""

    def __init__(self, wiring=None, name=None, model=None, date=None, debug=False):
        if wiring != None:
            self.wiring = wiring
        else:
            self.wiring = "ABCDEFGHIJKLMNOPQRSTUVWXYZ"
        self.name = name
        self.model = model
        self.date = date
        self.debug = debug

    def __setattr__(self, name, value):
        self.__dict__[name] = value

    def encipher(self, key):
        shift = ord(self.state) - ord("A")
        index = (ord(key) - ord("A")) % 26  # true index
        index = (index + shift) % 26  # actual connector hit
        letter = self.wiring[index]  # rotor letter generated
        out = chr(ord("A") + (ord(letter) - ord("A") + 26 - shift) % 26)  # actual output
        if self.debug:
            print(self.name,' [---] ', self.state,': ', key,' ', out, sep='');
        # return letter
        return out

    def __eq__(self, rotor):
        return self.name == rotor.name
    
    def __str__(self):
        """Pretty display."""
        return """
        Name: {}
        Model: {}
        Date: {}
        Wiring: {}""".format(
            self.name, self.model, self.date, self.wiring
        )

class Rotor:
    """Represents a rotor."""

    def __init__(
        self,
        wiring=None,
        notchs=None,
        name=None,
        model=None,
        date=None,
        state="A",
        ring="A",
        debug=False,
    ):
        """
        Initialization of the rotor.
        """
        if wiring != None:
            self.wiring = wiring
        else:
            self.wiring = "ABCDEFGHIJKLMNOPQRSTUVWXYZ"
        self.rwiring = ["0"] * 26
        for i in range(0, len(self.wiring)):
            self.rwiring[ord(self.wiring[i]) - ord("A")] = chr(ord("A") + i)
        if notchs != None:
            self.notchs = notchs
        else:
            self.notchs = ""
        self.name = name
        self.model = model
        self.date = date
        self.state = state
        self.ring = ring
        self.debug = debug

    def __setattr__(self, name, value):
        self.__dict__[name] = value
        if name == "wiring":
            self.rwiring = ["0"] * 26
            for i in range(0, len(self.wiring)):
                self.rwiring[ord(self.wiring[i]) - ord("A")] = chr(ord("A") + i)

    def encipher_right(self, key):
        shift = ord(self.state) - ord(self.ring) + 26
        index = (ord(key) - ord("A")) % 26  # true index
        index = (index + shift) % 26  # actual connector hit
        letter = self.wiring[index]  # rotor letter generated
        out = chr(ord("A") + (ord(letter) - ord("A") + 26 - shift) % 26)  # actual output
        if self.debug:
            print(self.name,' [>>>] ', self.state,': ', key,' ', out, sep='');
        return out

    def encipher_left(self, key):
        shift = ord(self.state) - ord(self.ring) + 26
        index = (ord(key) - ord("A")) % 26
        index = (index + shift) % 26
        # mapped output
        letter = self.rwiring[index]
        out = chr(ord("A") + (ord(letter) - ord("A") + 26 - shift) % 26)
        if self.debug:
            print(self.name,' [<<<] ', self.state,': ', key,' ', out, sep='');
        return out

    def notch(self, offset=1):
        r = chr((ord(self.state) + offset - ord("A")) % 26 + ord("A"))
        if self.debug:
            print(self.name,' [notch] ', self.state,' --> ', r, sep='');
        self.state = r
        notchnext = self.state in self.notchs
        return

    def is_in_turnover_pos(self):
        x = chr((ord(self.state) + 1 - ord("A")) % 26 + ord("A"))
        ret = x in self.notchs
        if self.debug:
            print(self.name,' [turnover] ? = ', ret, ', state = ', x, ', notchs = ', self.notchs, sep='');
        return ret

    def __eq__(self, rotor):
        return self.name == rotor.name

    def __str__(self):
        """
        Pretty display.
        """
        return """
        Name: {}
        Model: {}
        Date: {}
        Wiring: {}
        RWiring: {}
        State: {}""".format(
            self.name, self.model, self.date, self.wiring, ''.join(self.rwiring), self.state
        )


# 1924 Rotors
ROTOR_IC = Rotor(
    wiring="DMTWSILRUYQNKFEJCAZBPGXOHV", name="IC", model="Commercial Enigma A, B", date="1924",)
ROTOR_IIC = Rotor(wiring="HQZGPJTMOBLNCIFDYAWVEUSRKX",name="IIC",model="Commercial Enigma A, B",date="1924",)
ROTOR_IIIC = Rotor(wiring="UQNTLSZFMREHDPXKIBVYGJCWOA",name="IIIC",model="Commercial Enigma A, B",date="1924",)

# German Railway Rotors
ROTOR_GR_I = Rotor(wiring="JGDQOXUSCAMIFRVTPNEWKBLZYH",name="I",model="German Railway (Rocket)",date="7 February 1941",)
ROTOR_GR_II = Rotor(wiring="NTZPSFBOKMWRCJDIVLAEYUXHGQ",name="II",model="German Railway (Rocket)",date="7 February 1941",)
ROTOR_GR_III = Rotor(wiring="JVIUBHTCDYAKEQZPOSGXNRMWFL",name="III",model="German Railway (Rocket)",date="7 February 1941",)
ROTOR_GR_UKW = Reflector(wiring="QYHOGNECVPUZTFDJAXWMKISRBL",name="UTKW",model="German Railway (Rocket)",date="7 February 1941",)
ROTOR_GR_ETW = Rotor(wiring="QWERTZUIOASDFGHJKPYXCVBNML",name="ETW",model="German Railway (Rocket)",date="7 February 1941",)

# Swiss SONDER Rotors
ROTOR_I_SONDER = Rotor(wiring="VEOSIRZUJDQCKGWYPNXAFLTHMB",name="I-SONDER",model="Enigma Sonder",date="February 1939",)
ROTOR_II_SONDER = Rotor(wiring="UEMOATQLSHPKCYFWJZBGVXIDNR",name="II-SONDER",model="Enigma Sonder",date="February 1939",)
ROTOR_III_SONDER = Rotor(wiring="TZHXMBSIPNURJFDKEQVCWGLAOY",name="III-SONDER",model="Enigma Sonder",date="February 1939",)
ROTOR_UKW_SONDER = Reflector(wiring="CIAGSNDRBYTPZFULVHEKOQXWJM",name="UKW-SONDER",model="Enigma Sonder",date="February 1939",)
ROTOR_ETW_SONDER = Rotor(wiring="ABCDEFGHIJKLMNOPQRSTUVWXYZ",name="ETW-SONDER",model="Enigma Sonder",date="February 1939",)

# Enigma K Rotors
ROTOR_I_K = Rotor(wiring="LPGSZMHAEOQKVXRFYBUTNICJDW",notchs="G",name="I-K",model="Enigma K",date="February 1939",)
ROTOR_II_K = Rotor(wiring="SLVGBTFXJQOHEWIRZYAMKPCNDU",notchs="M",name="II-K",model="Enigma K",date="February 1939",)
ROTOR_III_K = Rotor(wiring="CJGDPSHKTURAWZXFMYNQOBVLIE",notchs="V",name="III-K",model="Enigma K",date="February 1939",)
ROTOR_UKW_K = Reflector(wiring="IMETCGFRAYSQBZXWLHKDVUPOJN",name="UKW-K",model="Enigma K",date="February 1939",)
ROTOR_ETW_K = Rotor(wiring="QWERTZUIOASDFGHJKPYXCVBNML",name="ETW-K",model="Enigma K",date="February 1939",)

# Enigma G Rotors, turnover: 'SUVWZABCEFGIKLOPQ', 'STVYZACDFGHKMNQ', 'UWXAEFHKMNR'
ROTOR_I_G = Rotor(wiring="LPGSZMHAEOQKVXRFYBUTNICJDW",notchs="ACDEHIJKMNOQSTWXY",name="I-G",model="Enigma G",date="",)
ROTOR_II_G = Rotor(wiring="SLVGBTFXJQOHEWIRZYAMKPCNDU",notchs="ABDGHIKLNOPSUVY",name="II-G",model="Enigma G",date="",)
ROTOR_III_G = Rotor(wiring="CJGDPSHKTURAWZXFMYNQOBVLIE",notchs="CEFIMNPSUVZ",name="III-G",model="Enigma G",date="",)
ROTOR_UKW_G = Reflector(wiring="IMETCGFRAYSQBZXWLHKDVUPOJN",name="UKW-G",model="Enigma G",date="",)
ROTOR_ETW_G = Rotor(wiring="QWERTZUIOASDFGHJKPYXCVBNML",name="ETW-G",model="Enigma G",date="",)

# Enigma R(ailway) Rotors
ROTOR_I_R = Rotor(wiring="JGDQOXUSCAMIFRVTPNEWKBLZYH",notchs="V",name="I-R",model="Enigma R",date="February 1939",)
ROTOR_II_R = Rotor(wiring="NTZPSFBOKMWRCJDIVLAEYUXHGQ",notchs="M",name="II-R",model="Enigma R",date="February 1939",)
ROTOR_III_R = Rotor(wiring="JVIUBHTCDYAKEQZPOSGXNRMWFL",notchs="G",name="III-R",model="Enigma R",date="February 1939",)
ROTOR_UKW_R = Reflector(wiring="QYHOGNECVPUZTFDJAXWMKISRBL",name="UKW-R",model="Enigma R",date="February 1939",)
ROTOR_ETW_R = Rotor(wiring="QWERTZUIOASDFGHJKPYXCVBNML",name="ETW-R",model="Enigma R",date="February 1939",)

# Swiss K Rotors
ROTOR_I_SK = Rotor(wiring="PEZUOHXSCVFMTBGLRINQJWAYDK",notchs="G",name="I-K",model="Swiss K",date="February 1939",)
ROTOR_II_SK = Rotor(wiring="ZOUESYDKFWPCIQXHMVBLGNJRAT",notchs="M",name="II-K",model="Swiss K",date="February 1939",)
ROTOR_III_SK = Rotor(wiring="EHRVXGAOBQUSIMZFLYNWKTPDJC",notchs="V",name="III-K",model="Swiss K",date="February 1939",)
ROTOR_UKW_SK = Reflector(wiring="IMETCGFRAYSQBZXWLHKDVUPOJN",name="UKW-K",model="Swiss K",date="February 1939",)
ROTOR_ETW_SK = Rotor(wiring="QWERTZUIOASDFGHJKPYXCVBNML",name="ETW-K",model="Swiss K",date="February 1939",)

# Enigma
ROTOR_I = Rotor(wiring="EKMFLGDQVZNTOWYHXUSPAIBRCJ",notchs="R",name="I",model="Enigma 1",date="1930",)
ROTOR_II = Rotor(wiring="AJDKSIRUXBLHWTMCQGZNPYFVOE",notchs="F",name="II",model="Enigma 1",date="1930",)
ROTOR_III = Rotor(wiring="BDFHJLCPRTXVZNYEIWGAKMUSQO",notchs="W",name="III",model="Enigma 1",date="1930",)
ROTOR_IV = Rotor(wiring="ESOVPZJAYQUIRHXLNFTGKDCMWB",notchs="K",name="IV",model="M3 Army",date="December 1938",)
ROTOR_V = Rotor(wiring="VZBRGITYUPSDNHLXAWMJQOFECK",notchs="A",name="V",model="M3 Army",date="December 1938",)
ROTOR_VI = Rotor(wiring="JPGVOUMFYQBENHZRDKASXLICTW",notchs="AN",name="VI",model="M3 & M4 Naval(February 1942)",date="1939",)
ROTOR_VII = Rotor(wiring="NZJHGRCXMYSWBOUFAIVLPEKQDT",notchs="AN",name="VII",model="M3 & M4 Naval(February 1942)",date="1939",)
ROTOR_VIII = Rotor(wiring="FKQHTLXOCBJSPDZRAMEWNIUYGV",notchs="AN",name="VIII",model="M3 & M4 Naval(February 1942)",date="1939",)

# misc & reflectors
ROTOR_Beta = Rotor(
    wiring="LEYJVCNIXWPBQMDRTAKZGFUHOS", name="Beta", model="M4 R2", date="Spring 1941"
)
ROTOR_Gamma = Rotor(
    wiring="FSOKANUERHMBTIYCWLQPZXVGJD", name="Gamma", model="M4 R2", date="Spring 1941"
)
ROTOR_Reflector_A = Reflector(wiring="EJMZALYXVBWFCRQUONTSPIKHGD", name="Reflector A")
ROTOR_Reflector_B = Reflector(wiring="YRUHQSLDPXNGOKMIEBFZCWVJAT", name="Reflector B")
ROTOR_Reflector_C = Reflector(wiring="FVPJIAOYEDRZXWGCTKUQSBNMHL", name="Reflector C")
ROTOR_Reflector_B_Thin = Reflector(
    wiring="ENKQAUYWJICOPBLMDXZVFTHRGS",
    name="Reflector_B_Thin",
    model="M4 R1 (M3 + Thin)",
    date="1940",
)
ROTOR_Reflector_C_Thin = Reflector(
    wiring="RDOBJNTKVEHMLFCWZAXGYIPSUQ",
    name="Reflector_C_Thin",
    model="M4 R1 (M3 + Thin)",
    date="1940",
)
ROTOR_ETW = Rotor(wiring="ABCDEFGHIJKLMNOPQRSTUVWXYZ", name="ETW", model="Enigma 1")

In [3]:
class Enigma:
    """Represents an Enigma machine.
    Initializes an Enigma machine with these arguments:
    - ref: reflector;
    - r1, r2, r3: rotors;
    - key: initial state of rotors;
    - plus: plugboard settings.
    """

    def __init__(self, ref, r1, r2, r3, key="AAA", plugs="", ring="AAA", notchs="", debug=False):
        """Initialization of the Enigma machine."""
        self.reflector = ref
        self.rotor1 = r1
        self.rotor2 = r2
        self.rotor3 = r3

        self.rotor1.state = key[0]
        self.rotor2.state = key[1]
        self.rotor3.state = key[2]
        self.rotor1.ring = ring[0]
        self.rotor2.ring = ring[1]
        self.rotor3.ring = ring[2]
        if len(notchs) >= 3:
            self.rotor1.notchs = notchs[0]
            self.rotor2.notchs = notchs[1]
            self.rotor3.notchs = notchs[2]
        self.reflector.state = "A"
        self.rotor1.debug = debug
        self.rotor2.debug = debug
        self.rotor3.debug = debug
        self.reflector.debug = debug
        self.debug = debug

        plugboard_settings = [(elem[0], elem[1]) for elem in plugs.split()]

        alpha = "ABCDEFGHIJKLMNOPQRSTUVWXYZ"
        alpha_out = [" "] * 26
        for i in range(len(alpha)):
            alpha_out[i] = alpha[i]
        for k, v in plugboard_settings:
            alpha_out[ord(k) - ord("A")] = v
            alpha_out[ord(v) - ord("A")] = k

        try:
            self.transtab = str.maketrans(alpha, "".join(alpha_out))
        except:
            # Python 2
            from string import maketrans

            self.transtab = maketrans(alpha, "".join(alpha_out))

    def encipher(self, plaintext_in):
        """Encrypt 'plaintext_in'."""
        ciphertext = ""
        plaintext_in_upper = plaintext_in.upper()
        plaintext = plaintext_in_upper.translate(self.transtab)
        for c in plaintext:

            # ignore non alphabetic char
            if not c.isalpha():
                ciphertext += c
            else:
                t = self.rotor1.encipher_right(c)
                t = self.rotor2.encipher_right(t)
                t = self.rotor3.encipher_right(t)
                t = self.reflector.encipher(t)
                t = self.rotor3.encipher_left(t)
                t = self.rotor2.encipher_left(t)
                t = self.rotor1.encipher_left(t)
                ciphertext += t

            if self.rotor1.is_in_turnover_pos():
                if self.rotor2.is_in_turnover_pos():
                    self.rotor3.notch()
                self.rotor2.notch()
            self.rotor1.notch()
            if self.debug:
                print('[states] : ', self.rotor1.state, self.rotor2.state, self.rotor3.state, sep='');

        res = ciphertext.translate(self.transtab)

        fres = ""
        for idx, char in enumerate(res):
            if plaintext_in[idx].islower():
                fres += char.lower()
            else:
                fres += char
        return fres
    
    def __str__(self):
        """Pretty display."""
        return """
        Reflector: {}
        Rotor 1: {}
        Rotor 2: {}
        Rotor 3: {}""".format(
            self.reflector, self.rotor1, self.rotor2, self.rotor3
        )

In [12]:
def msgToCipherByRing(message, ringset, show=False):
    # message: text to encipher
    # ringset: 3 letter ring value to have a cipher ending with
    # return all set of Key-Ring-Cipher tuple with the designated ringset at the end of encoded ciphers
    alphabets = "ABCDEFGHIJKLMNOPQRSTUVWXYZ"
    keys = []
    for x in alphabets:
        for y in alphabets:
            for z in alphabets:
                keys.append(x + y + z)
    
    ciphers = []
    for j in range(len(keys)):
        engine = Enigma(ROTOR_UKW_K, ROTOR_I_K, ROTOR_II_K, ROTOR_III_K, key=keys[j], ring=ringset, debug=False)
        decoded = engine.encipher(message).upper()
        idx = len(decoded) - 3
        s = decoded[idx:idx+3:1]
        if s == ringset:
            ciphers.append({'Key':keys[j], 'Ring':ringset, 'Message':message, 'Cipher':decoded})
    if len(ciphers) == 0:
        print("There is no Key-Ring-Cipher tuple with the designated ring, ", ringset)
    else:
        print("Key-Ring-Cipher tuple(s) found: ", str(len(ciphers)))
        if show:
            for x in ciphers: print(x)
    return ciphers

def findWordByFixedRing(ciphers, keywords, ringset, show=False):
    # ciphers: encrypted text to look for keywords
    # keywords: list of keywords looking for in ciphers
    # ringset: 3 letter ring value
    # return key-ring-cipher if decrypted text contains keywords by changing key values from AAA to ZZZ
    alphabets = "ABCDEFGHIJKLMNOPQRSTUVWXYZ"
    keys2 = []
    for x in alphabets:
        for y in alphabets:
            keys2.append(x + y)
            
    outputs = []
    for k in range(len(alphabets)):
        key1 = alphabets[k]
        #print("KEY 1: " + key1)
        for i in range(len(ciphers)):
            for j in range(len(keys2)):
                keyset = key1 + keys2[j]
                engine = Enigma(ROTOR_UKW_K, ROTOR_I_K, ROTOR_II_K, ROTOR_III_K, key=keyset, ring=ringset, debug=False)
                decoded = engine.encipher(ciphers[i])
                if isMatchFound(decoded, keywords):
                    outputs.append({'Key':keyset, 'Ring':ringset, 'Message':decoded, 'Cipher':ciphers[i]})
    if len(outputs) == 0:
        print("There is no Key-Ring-Cipher tuple for search word, ", ringset)
    else:
        print("Key-Ring-Cipher tuple(s) found: ", str(len(outputs)))
        if show:
            for x in outputs: print(x)
    return outputs

def getDecodedCiphersInFiles(ciphers, fixedring=""):
    # ciphers: a list of ciphers
    # ring is set to the last 3 lettets of the given cipher
    # changing key values from AAA to ZZZ, decrypt ciphers and save into file
    alphabets = "ABCDEFGHIJKLMNOPQRSTUVWXYZ"
    keys2 = []
    for x in alphabets:
        for y in alphabets:
            keys2.append(x + y)
    
    useFixedRing = False
    if len(fixedring) ==3:
        useFixedRing = True

    for k in range(len(alphabets)):
        key1 = alphabets[k]
        print("KEY 1: " + key1)
        for i in range(len(ciphers)):
            s = ciphers[i]
            ringset = s[len(s)-3:]
            if useFixedRing:
                ringset = fixedring.upper()
            parsedWords = []
            decList1 = []
            decList2 = []
            for j in range(len(keys2)):
                keyset = key1 + keys2[j]
                engine = Enigma(ROTOR_UKW_K, ROTOR_I_K, ROTOR_II_K, ROTOR_III_K, key=keyset, ring=ringset, debug=False)
                decoded = engine.encipher(ciphers[i]).upper()
                parsed = ' '.join(wordninja.split(decoded))
                s = str(parsed.count(' ') + 1)
                parsedWords.append(s.rjust(5, '0') + " - " + parsed + " : " + decoded + ": KEY - " + keyset + ": RING - " + ringset)
            if len(parsedWords):
                parsedWords.sort(reverse=False)
                if useFixedRing:
                    f = saveSortedListAsTextFile(parsedWords, ciphers[i] + "_r3_" + ringset, key1)
                else:
                    f = saveSortedListAsTextFile(parsedWords, ciphers[i] + "_r3", key1)
                print('File: ', f, ' saved...')
    return

def decryptCiphersFormatted(ciphers, keys, rings="", plugset=""):
    if len(plugset) == 0:
        print('Plug Board Settings: <none>' + '\n')
    else:
        print('Plug Board Settings: ' + plugset + '\n')

    for i in range(len(ciphers)):
        if len(rings) == 0:
            s = ciphers[i]
            ringset = s[len(s)-3:]
        else:
            ringset = rings
            
        engine = Enigma(ROTOR_UKW_K, ROTOR_I_K, ROTOR_II_K, ROTOR_III_K, key=keys, plugs=plugset, ring=ringset)
        decoded = engine.encipher(ciphers[i]).upper()
        parsed = ' '.join(wordninja.split(decoded))
        print(decoded.rjust(26, ' ') + " - K: " + keyset + " - R: " + ringset + ' -- ' + parsed.ljust(40, ' ') + " - C: " + s)
    return

In [5]:
findWordByFixedRing(["FEWGDRHDDEEUMFFTEEMJXZR"], ["BANK"], "AAA")

Key-Ring-Cipher tuple(s) found:  4


[{'Key': 'CVU',
  'Ring': 'AAA',
  'Message': 'MKXBFBANKODXXRTMOQTDHQD',
  'Cipher': 'FEWGDRHDDEEUMFFTEEMJXZR'},
 {'Key': 'FHG',
  'Ring': 'AAA',
  'Message': 'DWQRBZTZOADREBANKQTZQMD',
  'Cipher': 'FEWGDRHDDEEUMFFTEEMJXZR'},
 {'Key': 'JQE',
  'Ring': 'AAA',
  'Message': 'OUSTGOVPBANKIZMUBMHUVOG',
  'Cipher': 'FEWGDRHDDEEUMFFTEEMJXZR'},
 {'Key': 'KXB',
  'Ring': 'AAA',
  'Message': 'BANKUPCWXJKZEZPFSVJRLDI',
  'Cipher': 'FEWGDRHDDEEUMFFTEEMJXZR'}]

In [5]:
findWordByFixedRing(["FEWGDRHDDEEUMFFTEEMJXZR"], ["BANK"], "AAA")

Key-Ring-Cipher tuple(s) found:  4


[{'Key': 'CVU',
  'Ring': 'AAA',
  'Message': 'MKXBFBANKODXXRTMOQTDHQD',
  'Cipher': 'FEWGDRHDDEEUMFFTEEMJXZR'},
 {'Key': 'FHG',
  'Ring': 'AAA',
  'Message': 'DWQRBZTZOADREBANKQTZQMD',
  'Cipher': 'FEWGDRHDDEEUMFFTEEMJXZR'},
 {'Key': 'JQE',
  'Ring': 'AAA',
  'Message': 'OUSTGOVPBANKIZMUBMHUVOG',
  'Cipher': 'FEWGDRHDDEEUMFFTEEMJXZR'},
 {'Key': 'KXB',
  'Ring': 'AAA',
  'Message': 'BANKUPCWXJKZEZPFSVJRLDI',
  'Cipher': 'FEWGDRHDDEEUMFFTEEMJXZR'}]

In [62]:
ring = "XZR"
msg = "OUSTGOVPBANKIZMUBMHUVOG"
msgToCipherByRing(msg, ring, show=True)

Key-Ring-Cipher tuple(s) found:  2
{'Key': 'BUX', 'Ring': 'XZR', 'Message': 'OUSTGOVPBANKIZMUBMHUVOG', 'Cipher': 'CJJCQGOVVIYVEMBVVDDPXZR'}
{'Key': 'GPV', 'Ring': 'XZR', 'Message': 'OUSTGOVPBANKIZMUBMHUVOG', 'Cipher': 'FEWGDRHDDEEUMFFTEEMJXZR'}


[{'Key': 'BUX',
  'Ring': 'XZR',
  'Message': 'OUSTGOVPBANKIZMUBMHUVOG',
  'Cipher': 'CJJCQGOVVIYVEMBVVDDPXZR'},
 {'Key': 'GPV',
  'Ring': 'XZR',
  'Message': 'OUSTGOVPBANKIZMUBMHUVOG',
  'Cipher': 'FEWGDRHDDEEUMFFTEEMJXZR'}]

In [5]:
ring = "XZR"
msg = "HELLOWORLD"
msgToCipherByRing(msg, ring, show=True)

There is no Key-Ring-Cipher tuple with the designated ring,  XZR


[]

In [6]:
ring = "XXX"
msg = "HELLOWORLD"
msgToCipherByRing(msg, ring, show=True)

Key-Ring-Cipher tuple(s) found:  2
{'Key': 'CFY', 'Ring': 'XXX', 'Message': 'HELLOWORLD', 'Cipher': 'RBTUTBUXXX'}
{'Key': 'HDO', 'Ring': 'XXX', 'Message': 'HELLOWORLD', 'Cipher': 'WXMARVWXXX'}


[{'Key': 'CFY',
  'Ring': 'XXX',
  'Message': 'HELLOWORLD',
  'Cipher': 'RBTUTBUXXX'},
 {'Key': 'HDO',
  'Ring': 'XXX',
  'Message': 'HELLOWORLD',
  'Cipher': 'WXMARVWXXX'}]

In [8]:
# set RING to the fixedring
getDecodedCiphersInFiles(["helloworld"], fixedring="SRH")

KEY 1: A
File:  G:\DeepLearning\codes\goldbar\decoded\helloworld_r3_SRH\helloworld_r3_SRH_A.txt  saved...
KEY 1: B
File:  G:\DeepLearning\codes\goldbar\decoded\helloworld_r3_SRH\helloworld_r3_SRH_B.txt  saved...
KEY 1: C
File:  G:\DeepLearning\codes\goldbar\decoded\helloworld_r3_SRH\helloworld_r3_SRH_C.txt  saved...
KEY 1: D
File:  G:\DeepLearning\codes\goldbar\decoded\helloworld_r3_SRH\helloworld_r3_SRH_D.txt  saved...
KEY 1: E
File:  G:\DeepLearning\codes\goldbar\decoded\helloworld_r3_SRH\helloworld_r3_SRH_E.txt  saved...
KEY 1: F
File:  G:\DeepLearning\codes\goldbar\decoded\helloworld_r3_SRH\helloworld_r3_SRH_F.txt  saved...
KEY 1: G
File:  G:\DeepLearning\codes\goldbar\decoded\helloworld_r3_SRH\helloworld_r3_SRH_G.txt  saved...
KEY 1: H
File:  G:\DeepLearning\codes\goldbar\decoded\helloworld_r3_SRH\helloworld_r3_SRH_H.txt  saved...
KEY 1: I
File:  G:\DeepLearning\codes\goldbar\decoded\helloworld_r3_SRH\helloworld_r3_SRH_I.txt  saved...
KEY 1: J
File:  G:\DeepLearning\codes\goldbar\

In [6]:
# set RING to last 3 letters (fixedring="")
getDecodedCiphersInFiles(["helloworld"])

KEY 1: A
File:  G:\DeepLearning\codes\goldbar\decoded\helloworld_r3\helloworld_r3_A.txt  saved...
KEY 1: B
File:  G:\DeepLearning\codes\goldbar\decoded\helloworld_r3\helloworld_r3_B.txt  saved...
KEY 1: C
File:  G:\DeepLearning\codes\goldbar\decoded\helloworld_r3\helloworld_r3_C.txt  saved...
KEY 1: D
File:  G:\DeepLearning\codes\goldbar\decoded\helloworld_r3\helloworld_r3_D.txt  saved...
KEY 1: E
File:  G:\DeepLearning\codes\goldbar\decoded\helloworld_r3\helloworld_r3_E.txt  saved...
KEY 1: F
File:  G:\DeepLearning\codes\goldbar\decoded\helloworld_r3\helloworld_r3_F.txt  saved...
KEY 1: G
File:  G:\DeepLearning\codes\goldbar\decoded\helloworld_r3\helloworld_r3_G.txt  saved...
KEY 1: H
File:  G:\DeepLearning\codes\goldbar\decoded\helloworld_r3\helloworld_r3_H.txt  saved...
KEY 1: I
File:  G:\DeepLearning\codes\goldbar\decoded\helloworld_r3\helloworld_r3_I.txt  saved...
KEY 1: J
File:  G:\DeepLearning\codes\goldbar\decoded\helloworld_r3\helloworld_r3_J.txt  saved...
KEY 1: K
File:  G:\D

In [63]:
engine = Enigma(ROTOR_UKW_K, ROTOR_I_K, ROTOR_II_K, ROTOR_III_K, key="SNR", ring="XZX")
#print(engine)
secret = engine.encipher("OUSTGOVPBANKIZMUBMHUVOG")
print(secret)
#
engine = Enigma(ROTOR_UKW_K, ROTOR_I_K, ROTOR_II_K, ROTOR_III_K, key="SNR", ring="XZX")
text = engine.encipher(secret)
print(text)

SFUUKCULZUYSMHEJCKCBXZX
OUSTGOVPBANKIZMUBMHUVOG


In [18]:
keyset = "GPV"
ringset = "AAA" #"XZR"
text = "ABCDEFGHIJKLMNOPQRSTUVWXYZ" #"OUSTGOVPBANKIZMUBMHUVOG"
#
engine = Enigma(ROTOR_UKW_K, ROTOR_I_K, ROTOR_II_K, ROTOR_III_K, key=keyset, ring=ringset)
secret = engine.encipher(text)
print(text)
print(secret)
#
engine = Enigma(ROTOR_UKW_K, ROTOR_I_K, ROTOR_II_K, ROTOR_III_K, key=keyset, ring=ringset)
text = engine.encipher(secret)
print(text)

ABCDEFGHIJKLMNOPQRSTUVWXYZ
NSDZXUXNAOGSDGKLHEBBRIZMBI
ABCDEFGHIJKLMNOPQRSTUVWXYZ


In [17]:
cipher_shorts = [
    "FEWGDRHDDEEUMFFTEEMJXZR",#JQE
    "UGMNCBXCFLDEY", #SQE
    "HLMTAHGBGFNIV",#upside down-----
    "MLMTAHGBGFNIV",
    "ZUQUPNZN",
    "ABRYCTUGVZXUPB",
    "MVERZRLQDBHQ",
    "GKJFHYXODIE",
    "UGMNCBXCRLDEY",
    "UGMNCBXCFLDFY",
    "HFXPCQYZVATXAWIZPVE",#upside down-----
    "JKGFIJPMCWSAEK",#top down---
    "JKGFIJPMCWSAFK",
    "SKCDKJCDJCYQSZKTZJPXPWIRN",
    "MQOLCSJTLGAJOKBSSBOMUPCE",
    "RHZVIYQIYSXVNQXQWIOVWPJO",#top down---
    "YQHUDTABGALLOWLS",
    "VIOHIKNNGUAB",
    "KOWVRSRWTMLDH",
]

keyset = "IWA"
decryptCiphersFormatted(cipher_shorts, keyset)

print()
plug = "YQ HU DT AB GA LL OW LS"
decryptCiphersFormatted(cipher_shorts, keyset, plugset=plug)

Plug Board Settings: <none>

   WOMTQXTGZSNMKBPJNCXZEFP - K: IWA - R: XZR -- WO MT QX TGZ SN MK BP JN CX ZEF P        - C: FEWGDRHDDEEUMFFTEEMJXZR
             NHYJEOPEMACLM - K: IWA - R: DEY -- N HY JEO PE MAC LM                       - C: UGMNCBXCFLDEY
             XXOLZZENTTCJW - K: IWA - R: NIV -- XX O LZ ZEN TTC J W                      - C: HLMTAHGBGFNIV
             UXOLZZENTTCJW - K: IWA - R: NIV -- U XO LZ ZEN TTC J W                      - C: MLMTAHGBGFNIV
                  NBLGWIJF - K: IWA - R: NZN -- NBL GW IJ F                              - C: ZUQUPNZN
            JPOHUYMKEIILBJ - K: IWA - R: UPB -- J PO HUY M KEI IL BJ                     - C: ABRYCTUGVZXUPB
              IHVZQPEXEPKS - K: IWA - R: BHQ -- I HV Z QP EXE PK S                       - C: MVERZRLQDBHQ
               FJRQZISWQEZ - K: IWA - R: DIE -- F JR Q Z IS WQ EZ                        - C: GKJFHYXODIE
             NHYJEOPEQACLM - K: IWA - R: DEY -- N HY JEO PE QA CL M                      - C: UGMNCBXCRL

In [5]:
# notice that the maximum length of short ciphers is 25, less than 1 compared to the alphabets A to Z
cipher_shorts = [
    #"FEWGDRHDDEEUMFFTEEMJXZR",#JQE
    #"UGMNCBXCFLDEY", #SQE
    #"HLMTAHGBGFNIV",#upside down-----
    #"MLMTAHGBGFNIV",
    #"ZUQUPNZN",
    #"ABRYCTUGVZXUPB",
    #"MVERZRLQDBHQ",
    #"GKJFHYXODIE",
    #"UGMNCBXCRLDEY",
    #"UGMNCBXCFLDFY",
    #"HFXPCQYZVATXAWIZPVE",#upside down-----
    #"JKGFIJPMCWSAEK",#top down---
    #"SKCDKJCDJCYQSZKTZJPXPWIRN",
    #"MQOLCSJTLGAJOKBSSBOMUPCE",
    #"RHZVIYQIYSXVNQXQWIOVWPJO",#top down---
    #"YQHUDTABGALLOWLS",
    #"VIOHIKNNGUAB",
    #"KOWVRSRWTMLDH",
    #"XLYPISNANIRUSFTFWMIY",
    #
    #"YQVIOHIKNNGUABLS", # YQHUDTABGALLOWLS xor VIOHIKNNGUAB
    #"IZMUBMHU", # from OUSTGOVPBANK 'IZMUBMHU' VOG, double coding?
    #"UHMBUMZI", # reverse of IZMUBMHU
    #"JDSRKJSSBRVXVYHIQUL" # AMEND JDSRKJSSBRVXVYHIQUL from MQOLCSJTLGAJOKBSSBOMUPCE
    #"ALTYKDBJZ" # USE ALTYKDBJZ GOV DVDULIDUD from MQOLCSJTLGAJOKBSSBOMUPCE
    #"DVDULIDUD" # USE ALTYKDBJZ GOV DVDULIDUD from MQOLCSJTLGAJOKBSSBOMUPCE
    #"NLBRHQEM" # USE ACC NO NLBRHQEM from KOWVRSRWTMLDH
    #"VIOHIKNNGIWA" # change from VIOHIKNNGUAB with XOR ID IWA from ZUQUPNZN
    #"WXDYFXJFEXCBNS" # BAY WXDYFXJFEXCBNS TACKY LOT from SKCDKJCDJCYQSZKTZJPXPWIRN with Ring EFG
    #"WXDYFXJFEXCBN" # BAY WXDYFXJFEXCBN STACKY LOT from SKCDKJCDJCYQSZKTZJPXPWIRN with Ring EFG
    #"JDSIIEZKIUTARFHIQUL" # AMEND JDSIIEZKIUTARFHIQUL from MQOLCSJTLGAJOKBSSBOMUPCE with Ring AAA
    #"AEOUYBWQCUXKPQLTOGZRV" # GOV AEOUYBWQCUXKPQLTOGZRV from MQOLCSJTLGAJOKBSSBOMUPCE with Ring AAA
    #"ABRYCTUGVZXIWA", # EXOR ID IWA from ZUQUPNZN with Ring of FWT
    #"TWGYXTOAEWMYSXFH", # "QUIZ" TWGYXTOAEWMYSXFH from XLYPISNANIRUSFTFWMIY
    #"TWGYX", # "QUIZ" TWGYX TO AEWMYSXFH from XLYPISNANIRUSFTFWMIY
    #"AEWMYSXFH", # "QUIZ" TWGYX TO AEWMYSXFH from XLYPISNANIRUSFTFWMIY
    #"WDGZCPJOAPOCSXKBD", # "MSG" WDGZCPJOAPOCSXKBD from XLYPISNANIRUSFTFWMIY
    #"LFVSSHCNYHIO", # GAME LFVSSHCNYHIO ROOM from XLYPISNANIRUSFTFWMIY
    #"KDXIGIM", # KDXIGIM (FOR KDXIGIM OF WXAUTSM) from HFXPCQYZVATXAWIZPVE
    #"WXAUTSM", # WXAUTSM (FOR KDXIGIM OF WXAUTSM) from HFXPCQYZVATXAWIZPVE
    #"UIFEKSFGVWTZ", # KDXIGIM (US BERGE UIFEKSFGVWTZ) from HFXPCQYZVATXAWIZPVE
    #"YDZXUFCPC", # US BERGE GOV YDZXUFCPC from UIFEKSFGVWTZ (UIF12)
    #"CUFDLPT", # US BERGE ABIDE CUFDLPT from UIFEKSFGVWTZ (UIF12)
    #"NOCNNTTCLASES", # PAY NOCNNTTCLASES IF PASOLI from RHZVIYQIYSXVNQXQWIOVWPJO
    #"CNNTTCLASES", # PAY NO CNNTTCLASES IF PASOLI from RHZVIYQIYSXVNQXQWIOVWPJO
    #"PASOLI", # PAY NO CNNTTCLASES IF PASOLI from RHZVIYQIYSXVNQXQWIOVWPJO
    #"QIAHUNOQ", # KEY QIAHUNOQ UP from NOCNNTTCLASES of RHZVIYQIYSXVNQXQWIOVWPJO
    #"AAKAFFEYQ", # US AAKAFFEYQ UP from NOCNNTTCLASES of RHZVIYQIYSXVNQXQWIOVWPJO
    #"DBLTKA", # GOV DBLTKA from AAKAFFEYQ of NOCNNTTCLASES
    #"WBDGBZW", # BY WBDGBZW RIP ON from ABRYCTUGVZXUPB
    #"BWKOSOWFC", # KEY VS BWKOSOWFC from ABRYCTUGVZXUPB
    #"FCMULUDATG", # BY FCMULUDATG TO from ABRYCTUGVZXUPB
    #"SPIBLDZCQ", # REPAY SPIBLDZCQ from ABRYCTUGVZXUPB
    #"LENIELCLRHB", # GOV LENIELCLRHB TO from ABRYCTUGVZXUPB
    #"YQGKJFHYXODIELS", # YQ GKJFHYXODIE LS from MVERZRLQDBHQ GKJFHYXODIE
    #"LXGTPDIDAIJGBMWVP", # BY PAY QIN LXGTPDIDAIJGBMWVP from SKCDKJCDJCYQSZKTZJPXPWIRN with IRN
    #"LFDZQDIDAIJGBMWVP", # BY PAY QIN LFDZQDIDAIJGBMWVP from SKCDKJCDJCYQSZKTZJPXPWIRN with EFG
    #"DSPYOZ", # IF USE DSPYOZ from GKJFHYXODIE
    "LUQXGUMS", # LUQXGUMS SIGCO from UGMNCBXCFLDEY with AZH
    "THEIWKSRQT", # WAR THEIWKSRQT from UGMNCBXCFLDEY with AZH
    "FFYAWQVXT", # GOLD FFYAWQVXT from UGMNCBXCFLDEY with AZH
    "DIROSDOLI", # CASH DIROSDOLI from UGMNCBXCFLDEY with AZH
]

# start with 'H', do 'G' at the end since it rotates the second rotor
alphabets = "ABCDEFGHIJKLMNOPQRSTUVWXYZ"
alpha1 = alphabets
alpha2 = alphabets
alpha3 = alphabets
keys2 = []
for x in alpha2:
    for y in alpha3:
        keys2.append(x + y)

for k in range(len(alpha1)):
    key1 = alpha1[k]
    print("KEY 1: " + key1)
    for i in range(len(cipher_shorts)):
        s = cipher_shorts[i]
        ringset = s[len(s)-3:]
        parsedWords = []
        decList1 = []
        decList2 = []
        for j in range(len(keys2)):
            keyset = key1 + keys2[j]
            #
            engine = Enigma(ROTOR_UKW_K, ROTOR_I_K, ROTOR_II_K, ROTOR_III_K, key=keyset, ring=ringset, debug=False)
            decoded = engine.encipher(cipher_shorts[i]).upper()
            parsed = ' '.join(wordninja.split(decoded))
            s = str(parsed.count(' ') + 1)
            parsedWords.append(s.rjust(5, '0') + " - " + parsed + " : " + decoded + ": KEY - " + keyset + ": RING - " + ringset)
        #
        if len(parsedWords):
            parsedWords.sort(reverse=False)
            saveSortedListAsTextFile(parsedWords, cipher_shorts[i] + "_123_ninja_r3", key1)


KEY 1: A
KEY 1: B
KEY 1: C
KEY 1: D
KEY 1: E
KEY 1: F
KEY 1: G
KEY 1: H
KEY 1: I
KEY 1: J
KEY 1: K
KEY 1: L
KEY 1: M
KEY 1: N
KEY 1: O
KEY 1: P
KEY 1: Q
KEY 1: R
KEY 1: S
KEY 1: T
KEY 1: U
KEY 1: V
KEY 1: W
KEY 1: X
KEY 1: Y
KEY 1: Z


In [5]:
# 8/31/2024 - use specific Ring for decipherment instead of R3 letters
# notice that the maximum length of short ciphers is 25, less than 1 compared to the alphabets A to Z
cipher_shorts = [
    #"FEWGDRHDDEEUMFFTEEMJXZR",#JQE
    #"UGMNCBXCFLDEY", #SQE
    #"HLMTAHGBGFNIV",#upside down-----
    #"MLMTAHGBGFNIV",
    #"ZUQUPNZN",
    #"ABRYCTUGVZXUPB",
    #"MVERZRLQDBHQ",
    #"GKJFHYXODIE",
    "UGMNCBXCRLDEY",
    #"UGMNCBXCFLDFY",
    #"HFXPCQYZVATXAWIZPVE",#upside down-----
    #"JKGFIJPMCWSAEK",#top down---
    #"JKGFIJPMCWSAFK",
    #"SKCDKJCDJCYQSZKTZJPXPWIRN",
    #"MQOLCSJTLGAJOKBSSBOMUPCE",
    #"RHZVIYQIYSXVNQXQWIOVWPJO",#top down---
    #"YQHUDTABGALLOWLS",
    #"VIOHIKNNGUAB",
    #"KOWVRSRWTMLDH",
    #"XLYPISNANIRUSFTFWMIY",
    #
    #"YQVIOHIKNNGUABLS", # YQHUDTABGALLOWLS xor VIOHIKNNGUAB
    #"IZMUBMHU", # from OUSTGOVPBANK 'IZMUBMHU' VOG, double coding?
    #"UHMBUMZI", # reverse of IZMUBMHU
    #"JKGFIJPMCWSAEK", # with FWT from CONTACT FWT DEWVRY (YQVIOHIKNNGUABLS)
    #"ZUQUPNZN", # with FWT from CONTACT FWT DEWVRY (YQVIOHIKNNGUABLS)
    #"YQGKJFHYXODIELS", # YQ GKJFHYXODIE LS from MVERZRLQDBHQ GKJFHYXODIE
]

# start with 'H', do 'G' at the end since it rotates the second rotor
alphabets = "ABCDEFGHIJKLMNOPQRSTUVWXYZ"
alpha1 = alphabets
alpha2 = alphabets
alpha3 = alphabets
keys2 = []
for x in alpha2:
    for y in alpha3:
        keys2.append(x + y)

#fixedring = "SRH" # from TWO MSG XOR 'SRH'
#fixedring = "FWT" # from CONTACT FWT DEWVRY (YQVIOHIKNNGUABLS)
#fixedring = "EFG" # from ACT DMZ EFG SIG CT (JKGFIJPMCWSAEK)
#fixedring = "GMH" # from PIER GMH OVAL SAD (WXDYFXJFEXCBNS)
#fixedring = "ZZV" # from BLOBS NIEM RD ZZV (WXDYFXJFEXCBNS)
#fixedring = "BEW" # from ABV BEW JR FF MAM (UGMNCBXCRLDEY)
fixedring = "AZH" # from AZH LATE TOLD (GKJFHYXODIE > GKJFHYXOIWA)

for k in range(len(alpha1)):
    key1 = alpha1[k]
    print("KEY 1: " + key1)
    for i in range(len(cipher_shorts)):
        s = cipher_shorts[i]
        ringset = fixedring #s[len(s)-3:]
        parsedWords = []
        decList1 = []
        decList2 = []
        for j in range(len(keys2)):
            keyset = key1 + keys2[j]
            #
            engine = Enigma(ROTOR_UKW_K, ROTOR_I_K, ROTOR_II_K, ROTOR_III_K, key=keyset, ring=ringset, debug=False)
            decoded = engine.encipher(cipher_shorts[i]).upper()
            parsed = ' '.join(wordninja.split(decoded))
            s = str(parsed.count(' ') + 1)
            parsedWords.append(s.rjust(5, '0') + " - " + parsed + " : " + decoded + ": KEY - " + keyset + ": RING - " + ringset)
        #
        if len(parsedWords):
            parsedWords.sort(reverse=False)
            saveSortedListAsTextFile(parsedWords, cipher_shorts[i] + "_123_ninja_r3_" + fixedring, key1)


KEY 1: A
KEY 1: B
KEY 1: C
KEY 1: D
KEY 1: E
KEY 1: F
KEY 1: G
KEY 1: H
KEY 1: I
KEY 1: J
KEY 1: K
KEY 1: L
KEY 1: M
KEY 1: N
KEY 1: O
KEY 1: P
KEY 1: Q
KEY 1: R
KEY 1: S
KEY 1: T
KEY 1: U
KEY 1: V
KEY 1: W
KEY 1: X
KEY 1: Y
KEY 1: Z


In [6]:
# 8/27/2024 - use 3 letters for rings out of a given cipher and find exact matches
# notice that the maximum length of short ciphers is 25, less than 1 compared to the alphabets A to Z
cipher_shorts = [
    #"FEWGDRHDDEEUMFFTEEMJXZR",#JQE
    #"UGMNCBXCFLDEY", #SQE
    #"HLMTAHGBGFNIV",#upside down-----
    #"ZUQUPNZN",
    #"ABRYCTUGVZXUPB",
    #"MVERZRLQDBHQ",
    #"GKJFHYXODIE",
    #"UGMNCBXCRLDEY",
    #"UGMNCBXCFLDFY",
    "HFXPCQYZVATXAWIZPVE",#upside down-----
    #"JKGFIJPMCWSAEK",#top down---
    #"SKCDKJCDJCYQSZKTZJPXPWIRN",
    #"MQOLCSJTLGAJOKBSSBOMUPCE",
    #"RHZVIYQIYSXVNQXQWIOVWPJO",#top down---
    #"YQHUDTABGALLOWLS",
    #"VIOHIKNNGUAB",
    #"KOWVRSRWTMLDH",
]
keywords = [
    #"KEY",
    #"BANK", #FEWGDRHDDEEUMFFTEEMJXZR
    #"OUSTGOV" #FEWGDRHDDEEUMFFTEEMJXZR
    #"TWOMSG", #MVERZRLQDBHQ
    #"OKFLY", #MVERZRLQDBHQ
    #"HEACTIN", #MVERZRLQDBHQ
    #"EXPMOAUG", #MQOLCSJTLGAJOKBSSBOMUPCE
    #"ALLOR", #ZUQUPNZN
    #"ACCNO", #KOWVRSRWTMLDH
    #"QUERY",
    #"USA",
    #"TXNAME",
    #"QUARZ",
    #"LINDE", #HLMTAHGBGFNIV
    #"SIGCO",
    "USBERGE",
]
# start with 'H', do 'G' at the end since it rotates the second rotor
alphabets = "ABCDEFGHIJKLMNOPQRSTUVWXYZ"
alpha1 = alphabets
alpha2 = alphabets
alpha3 = alphabets
keys2 = []
for x in alpha2:
    for y in alpha3:
        keys2.append(x + y)

for kk in range(len(alpha1)):
    key1 = alpha1[kk]
    print("KEY 1: " + key1)
    for i in range(len(cipher_shorts)):
        # forward
        print("--- Forward ---")
        ringlist = getListof3Letters(cipher_shorts[i])
        if kk == 0: print(ringlist)
        for k in range(len(ringlist)):
            for j in range(len(keys2)):
                keyset = key1 + keys2[j]
                engine = Enigma(ROTOR_UKW_K, ROTOR_I_K, ROTOR_II_K, ROTOR_III_K, key=keyset, ring=ringlist[k], debug=False)
                decoded = engine.encipher(cipher_shorts[i]).upper()
                if isMatchFound(decoded, keywords, andAll=False):
                    t1 = cipher_shorts[i] + ": " + decoded + ' -key- ' + keyset + ' -ring- ' + ringlist[k]
                    print(t1, sep="")
        # reverse
        print("--- Reverse ---")
        s = cipher_shorts[i]
        ringlist = getListof3Letters(s[::-1])
        if kk == 0: print(ringlist)
        for k in range(len(ringlist)):
            for j in range(len(keys2)):
                keyset = key1 + keys2[j]
                engine = Enigma(ROTOR_UKW_K, ROTOR_I_K, ROTOR_II_K, ROTOR_III_K, key=keyset, ring=ringlist[k], debug=False)
                decoded = engine.encipher(cipher_shorts[i]).upper()
                if isMatchFound(decoded, keywords, andAll=False):
                    t1 = cipher_shorts[i] + ": " + decoded + ' -key- ' + keyset + ' -ring- ' + ringlist[k]
                    print(t1, sep="")


KEY 1: A
--- Forward ---
['HFX', 'FXP', 'XPC', 'PCQ', 'CQY', 'QYZ', 'YZV', 'ZVA', 'VAT', 'ATX', 'TXA', 'XAW', 'AWI', 'WIZ', 'IZP', 'ZPV', 'PVE']
--- Reverse ---
['EVP', 'VPZ', 'PZI', 'ZIW', 'IWA', 'WAX', 'AXT', 'XTA', 'TAV', 'AVZ', 'VZY', 'ZYQ', 'YQC', 'QCP', 'CPX', 'PXF', 'XFH']
KEY 1: B
--- Forward ---
--- Reverse ---
KEY 1: C
--- Forward ---
--- Reverse ---
KEY 1: D
--- Forward ---
--- Reverse ---
KEY 1: E
--- Forward ---
--- Reverse ---
KEY 1: F
--- Forward ---
--- Reverse ---
KEY 1: G
--- Forward ---
--- Reverse ---
KEY 1: H
--- Forward ---
--- Reverse ---
KEY 1: I
--- Forward ---
HFXPCQYZVATXAWIZPVE: USBERGEUIFEKSFGVWTZ -key- IKO -ring- TXA
--- Reverse ---
HFXPCQYZVATXAWIZPVE: USBERGEUIFEKSFGVWTZ -key- INJ -ring- TAV
KEY 1: J
--- Forward ---
--- Reverse ---
KEY 1: K
--- Forward ---
HFXPCQYZVATXAWIZPVE: USBERGEUIFEKSFGVWTZ -key- KNH -ring- VAT
--- Reverse ---
HFXPCQYZVATXAWIZPVE: USBERGEUIFEKSFGVWTZ -key- KCN -ring- VPZ
HFXPCQYZVATXAWIZPVE: USBERGEUIFEKSFGVWTZ -key- KMM -ring- VZY

In [7]:
cipher_shorts = [
    #"FEWGDRHDDEEUMFFTEEMJXZR",#JQE
    #"UGMNCBXCFLDEY", #SQE
    #"HLMTAHGBGFNIV",#upside down-----
    #"MLMTAHGBGFNIV",
    #"ZUQUPNZN",
    #"ABRYCTUGVZXUPB",
    #"MVERZRLQDBHQ",
    #"GKJFHYXODIE",
    #"UGMNCBXCRLDEY",
    #"UGMNCBXCFLDFY",
    "HFXPCQYZVATXAWIZPVE",#upside down-----
    #"JKGFIJPMCWSAEK",#top down---
    #"JKGFIJPMCWSAFK",
    #"SKCDKJCDJCYQSZKTZJPXPWIRN",
    #"MQOLCSJTLGAJOKBSSBOMUPCE",
    #"RHZVIYQIYSXVNQXQWIOVWPJO",#top down---
    #"YQHUDTABGALLOWLS",
    #"VIOHIKNNGUAB",
    #"KOWVRSRWTMLDH",
]
keywords = [
    #"KEY",
    #"BANK",
    #"GOV",
    #"OUSTGOV",
    #"QUERY",
    #"USA",
    #"TXNAME",
    #"QUARZ",
    #"THIEF",
    #"AMEND",
    #"BYPAY",
    #"IFUSED",
    "USBERGE",
]
alphabets = "ABCDEFGHIJKLMNOPQRSTUVWXYZ"
alpha1 = alphabets
alpha2 = alphabets
alpha3 = alphabets
keys2 = []
for x in alpha2:
    for y in alpha3:
        keys2.append(x + y)

#plug = "YQ HU DT AB GA LL OW LS"
plug = ""
ringset = "SRH" #"DIE" #"XZR"

for k in range(len(alpha1)):
    key1 = alpha1[k]
    print("KEY 1: " + key1)
    for i in range(len(cipher_shorts)):
        for j in range(len(keys2)):
            keyset = key1 + keys2[j]
            engine = Enigma(ROTOR_UKW_K, ROTOR_I_K, ROTOR_II_K, ROTOR_III_K, key=keyset, plugs=plug, ring=ringset, debug=False)
            decoded = engine.encipher(cipher_shorts[i]).upper()
            for kk in range(len(keywords)):
                if isMatchFound(decoded, keywords, andAll=False):
                    t1 = cipher_shorts[i] + ": " + decoded + " keyword: " + keywords[kk] + ' -key- ' + keyset + ' -ring- ' + ringset
                    print(t1, sep="")


KEY 1: A
KEY 1: B
KEY 1: C
KEY 1: D
KEY 1: E
KEY 1: F
KEY 1: G
KEY 1: H
HFXPCQYZVATXAWIZPVE: USBERGEUIFEKSFGVWTZ keyword: USBERGE -key- HEV -ring- SRH
KEY 1: I
KEY 1: J
KEY 1: K
KEY 1: L
KEY 1: M
KEY 1: N
KEY 1: O
KEY 1: P
KEY 1: Q
KEY 1: R
KEY 1: S
KEY 1: T
KEY 1: U
KEY 1: V
KEY 1: W
KEY 1: X
KEY 1: Y
KEY 1: Z


In [18]:
# 8/29/2024 - fix keyset and ringset, change notches
cipher_shorts = [
    "FEWGDRHDDEEUMFFTEEMJXZR",#JQE
    #"UGMNCBXCFLDEY", #SQE
    #"HLMTAHGBGFNIV",#upside down-----
    #"MLMTAHGBGFNIV",
    #"ZUQUPNZN",
    #"ABRYCTUGVZXUPB",
    #"MVERZRLQDBHQ",
    #"GKJFHYXODIE",
    #"UGMNCBXCRLDEY",
    #"UGMNCBXCFLDFY",
    #"HFXPCQYZVATXAWIZPVE",#upside down-----
    #"JKGFIJPMCWSAEK",#top down---
    #"JKGFIJPMCWSAFK",
    #"SKCDKJCDJCYQSZKTZJPXPWIRN",
    #"MQOLCSJTLGAJOKBSSBOMUPCE",
    #"RHZVIYQIYSXVNQXQWIOVWPJO",#top down---
    #"YQHUDTABGALLOWLS",
    #"VIOHIKNNGUAB",
    #"KOWVRSRWTMLDH",
]
keywords = [
    #"KEY",
    #"BANK",
    #"GOV",
    #"OUSTGOV"
    "OUSTGOVPBANKIZMUBMHUVOG"
    #"QUERY"
    #"USA"
    #"TXNAME"
    #"QUARZ"
    #"THIEF"
]
alphabets = "ABCDEFGHIJKLMNOPQRSTUVWXYZ"
alpha1 = alphabets
alpha2 = alphabets
alpha3 = alphabets
keys2 = []
for x in alpha2:
    for y in alpha3:
        keys2.append(x + y)

#plug = "YQ HU DT AB GA LL OW LS"
plug = ""
ringset = "XZR"
zerokey = "GPV"

for k in range(len(alpha1)):
    key1 = alpha1[k]
    print("KEY 1: " + key1)
    for i in range(len(cipher_shorts)):
        for j in range(len(keys2)):
            keyset = key1 + keys2[j]
            engine = Enigma(ROTOR_UKW_K, ROTOR_I_K, ROTOR_II_K, ROTOR_III_K, key=zerokey, plugs=plug, ring=ringset, notchs=keyset, debug=False)
            decoded = engine.encipher(cipher_shorts[i]).upper()
            for kk in range(len(keywords)):
                if isMatchFound(decoded, keywords, andAll=False):
                    t1 = cipher_shorts[i] + ": " + decoded + " keyword: " + keywords[kk] + ' -notch- ' + keyset + ' -zerokey- ' + zerokey + ' -ring- ' + ringset
                    print(t1, sep="")


KEY 1: A
KEY 1: B
KEY 1: C
KEY 1: D
FEWGDRHDDEEUMFFTEEMJXZR: OUSTGOVPBANKIZMUBMHUVOG keyword: OUSTGOVPBANKIZMUBMHUVOG -notch- DAA -key- GPV -ring- XZR
FEWGDRHDDEEUMFFTEEMJXZR: OUSTGOVPBANKIZMUBMHUVOG keyword: OUSTGOVPBANKIZMUBMHUVOG -notch- DAB -key- GPV -ring- XZR
FEWGDRHDDEEUMFFTEEMJXZR: OUSTGOVPBANKIZMUBMHUVOG keyword: OUSTGOVPBANKIZMUBMHUVOG -notch- DAC -key- GPV -ring- XZR
FEWGDRHDDEEUMFFTEEMJXZR: OUSTGOVPBANKIZMUBMHUVOG keyword: OUSTGOVPBANKIZMUBMHUVOG -notch- DAD -key- GPV -ring- XZR
FEWGDRHDDEEUMFFTEEMJXZR: OUSTGOVPBANKIZMUBMHUVOG keyword: OUSTGOVPBANKIZMUBMHUVOG -notch- DAE -key- GPV -ring- XZR
FEWGDRHDDEEUMFFTEEMJXZR: OUSTGOVPBANKIZMUBMHUVOG keyword: OUSTGOVPBANKIZMUBMHUVOG -notch- DAF -key- GPV -ring- XZR
FEWGDRHDDEEUMFFTEEMJXZR: OUSTGOVPBANKIZMUBMHUVOG keyword: OUSTGOVPBANKIZMUBMHUVOG -notch- DAG -key- GPV -ring- XZR
FEWGDRHDDEEUMFFTEEMJXZR: OUSTGOVPBANKIZMUBMHUVOG keyword: OUSTGOVPBANKIZMUBMHUVOG -notch- DAH -key- GPV -ring- XZR
FEWGDRHDDEEUMFFTEEMJXZR: OUSTGOVPBANKIZMUBMH

FEWGDRHDDEEUMFFTEEMJXZR: OUSTGOVPBANKIZMUBMHUVOG keyword: OUSTGOVPBANKIZMUBMHUVOG -notch- ETP -key- GPV -ring- XZR
FEWGDRHDDEEUMFFTEEMJXZR: OUSTGOVPBANKIZMUBMHUVOG keyword: OUSTGOVPBANKIZMUBMHUVOG -notch- ETQ -key- GPV -ring- XZR
FEWGDRHDDEEUMFFTEEMJXZR: OUSTGOVPBANKIZMUBMHUVOG keyword: OUSTGOVPBANKIZMUBMHUVOG -notch- ETR -key- GPV -ring- XZR
FEWGDRHDDEEUMFFTEEMJXZR: OUSTGOVPBANKIZMUBMHUVOG keyword: OUSTGOVPBANKIZMUBMHUVOG -notch- ETS -key- GPV -ring- XZR
FEWGDRHDDEEUMFFTEEMJXZR: OUSTGOVPBANKIZMUBMHUVOG keyword: OUSTGOVPBANKIZMUBMHUVOG -notch- ETT -key- GPV -ring- XZR
FEWGDRHDDEEUMFFTEEMJXZR: OUSTGOVPBANKIZMUBMHUVOG keyword: OUSTGOVPBANKIZMUBMHUVOG -notch- ETU -key- GPV -ring- XZR
FEWGDRHDDEEUMFFTEEMJXZR: OUSTGOVPBANKIZMUBMHUVOG keyword: OUSTGOVPBANKIZMUBMHUVOG -notch- ETV -key- GPV -ring- XZR
FEWGDRHDDEEUMFFTEEMJXZR: OUSTGOVPBANKIZMUBMHUVOG keyword: OUSTGOVPBANKIZMUBMHUVOG -notch- ETW -key- GPV -ring- XZR
FEWGDRHDDEEUMFFTEEMJXZR: OUSTGOVPBANKIZMUBMHUVOG keyword: OUSTGOVPBANKIZMUBMHUVO

FEWGDRHDDEEUMFFTEEMJXZR: OUSTGOVPBANKIZMUBMHUVOG keyword: OUSTGOVPBANKIZMUBMHUVOG -notch- GKI -key- GPV -ring- XZR
FEWGDRHDDEEUMFFTEEMJXZR: OUSTGOVPBANKIZMUBMHUVOG keyword: OUSTGOVPBANKIZMUBMHUVOG -notch- GKJ -key- GPV -ring- XZR
FEWGDRHDDEEUMFFTEEMJXZR: OUSTGOVPBANKIZMUBMHUVOG keyword: OUSTGOVPBANKIZMUBMHUVOG -notch- GKK -key- GPV -ring- XZR
FEWGDRHDDEEUMFFTEEMJXZR: OUSTGOVPBANKIZMUBMHUVOG keyword: OUSTGOVPBANKIZMUBMHUVOG -notch- GKL -key- GPV -ring- XZR
FEWGDRHDDEEUMFFTEEMJXZR: OUSTGOVPBANKIZMUBMHUVOG keyword: OUSTGOVPBANKIZMUBMHUVOG -notch- GKM -key- GPV -ring- XZR
FEWGDRHDDEEUMFFTEEMJXZR: OUSTGOVPBANKIZMUBMHUVOG keyword: OUSTGOVPBANKIZMUBMHUVOG -notch- GKN -key- GPV -ring- XZR
FEWGDRHDDEEUMFFTEEMJXZR: OUSTGOVPBANKIZMUBMHUVOG keyword: OUSTGOVPBANKIZMUBMHUVOG -notch- GKO -key- GPV -ring- XZR
FEWGDRHDDEEUMFFTEEMJXZR: OUSTGOVPBANKIZMUBMHUVOG keyword: OUSTGOVPBANKIZMUBMHUVOG -notch- GKP -key- GPV -ring- XZR
FEWGDRHDDEEUMFFTEEMJXZR: OUSTGOVPBANKIZMUBMHUVOG keyword: OUSTGOVPBANKIZMUBMHUVO

KEY 1: I
KEY 1: J
KEY 1: K
KEY 1: L
KEY 1: M
KEY 1: N
KEY 1: O
KEY 1: P
KEY 1: Q
KEY 1: R
KEY 1: S
KEY 1: T
KEY 1: U
KEY 1: V
KEY 1: W
KEY 1: X
KEY 1: Y
KEY 1: Z


In [9]:
# 8/29/2024 - fix keyset and ringset, change notches
cipher_shorts = [
    #"FEWGDRHDDEEUMFFTEEMJXZR",#JQE
    #"UGMNCBXCFLDEY", #SQE
    #"HLMTAHGBGFNIV",#upside down-----
    #"MLMTAHGBGFNIV",
    #"ZUQUPNZN",
    #"ABRYCTUGVZXUPB",
    #"MVERZRLQDBHQ",
    #"GKJFHYXODIE",
    #"UGMNCBXCRLDEY",
    #"UGMNCBXCFLDFY",
    #"HFXPCQYZVATXAWIZPVE",#upside down-----
    #"JKGFIJPMCWSAEK",#top down---
    #"JKGFIJPMCWSAFK",
    #"SKCDKJCDJCYQSZKTZJPXPWIRN",
    "MQOLCSJTLGAJOKBSSBOMUPCE",
    #"RHZVIYQIYSXVNQXQWIOVWPJO",#top down---
    #"YQHUDTABGALLOWLS",
    #"VIOHIKNNGUAB",
    #"KOWVRSRWTMLDH",
]
keywords = [
    #"KEY",
    #"BANK",
    #"GOV",
    #"OUSTGOV",
    #"OUSTGOVPBANKIZMUBMHUVOG",
    #"QUERY",
    #"USA",
    #"TXNAME",
    #"QUARZ",
    #"THIEF",
    "AMEND",
]
alphabets = "ABCDEFGHIJKLMNOPQRSTUVWXYZ"
alpha1 = alphabets
alpha2 = alphabets
alpha3 = alphabets
keys2 = []
for x in alpha2:
    for y in alpha3:
        keys2.append(x + y)

#plug = "YQ HU DT AB GA LL OW LS"
plug = ""
ringset = "AAA"

for k in range(len(alpha1)):
    key1 = alpha1[k]
    print("KEY 1: " + key1)
    for i in range(len(cipher_shorts)):
        for j in range(len(keys2)):
            keyset = key1 + keys2[j]
            engine = Enigma(ROTOR_UKW_K, ROTOR_I_K, ROTOR_II_K, ROTOR_III_K, key=keyset, plugs=plug, ring=ringset, debug=False)
            decoded = engine.encipher(cipher_shorts[i]).upper()
            for kk in range(len(keywords)):
                if isMatchFound(decoded, keywords, andAll=False):
                    t1 = cipher_shorts[i] + ": " + decoded + " keyword: " + keywords[kk] + ' -key- ' + keyset + ' -ring- ' + ringset
                    print(t1, sep="")


KEY 1: A
KEY 1: B
KEY 1: C
KEY 1: D
KEY 1: E
KEY 1: F
KEY 1: G
KEY 1: H
KEY 1: I
KEY 1: J
KEY 1: K
KEY 1: L
KEY 1: M
KEY 1: N
KEY 1: O
KEY 1: P
KEY 1: Q
KEY 1: R
KEY 1: S
KEY 1: T
KEY 1: U
KEY 1: V
KEY 1: W
KEY 1: X
KEY 1: Y
MQOLCSJTLGAJOKBSSBOMUPCE: AMENDJDSIIEZKIUTARFHIQUL keyword: AMEND -key- YXT -ring- AAA
KEY 1: Z


In [5]:
# notice that the maximum length of short ciphers is 25, less than 1 compared to the alphabets A to Z
cipher_shorts = [
    #"FEWGDRHDDEEUMFFTEEMJXZR",
    #"UGMNCBXCFLDBEY",
    #"GKJFHYXODIE",
    
    #"ABRYCTUGVZXUPB",
    #"HFXPCQYZVATXAWIZPVE",
    #"HLMTAHGBGFNIV",
    #"JKGFIJPMCWSAEK",
    #"KOWVRSRKWTMLDH",
    #"MQOLCSJTLGAJOKBSSBOMUPCE",
    "MVERZRLQDBHQ",
    #"RHZVIYQIYSXVNQXQWIOVWPJO",
    #"SKCDKJCDJCYQSZKTZJPXPWIRN",
    #"VIOHIKNNGUAB",
    #"YQHUDTABGALLOWLS",
    #"ZUQUPNZN"
]

# start with 'H', do 'G' at the end since it rotates the second rotor
alphabets = "ABCDEFGHIJKLMNOPQRSTUVWXYZ"
alpha1 = alphabets
alpha2 = alphabets
alpha3 = alphabets
keys2 = []
for x in alpha2:
    for y in alpha3:
        keys2.append(x + y)

for k in range(len(alpha1)):
    key1 = alpha1[k]
    print("KEY 1: " + key1)
    for i in range(len(cipher_shorts)):
        parsedWords = []
        decList1 = []
        decList2 = []
        for j in range(len(keys2)):
            keyset = key1 + keys2[j]
            #
            engine = Enigma(ROTOR_UKW_K, ROTOR_I_K, ROTOR_III_K, ROTOR_II_K, key=keyset, ring='AAA', debug=False)
            decoded = engine.encipher(cipher_shorts[i]).upper()
            parsed = ' '.join(wordninja.split(decoded))
            parsedWords.append(parsed + " : " + decoded + ": " + keyset)
        #
        if len(parsedWords):
            parsedWords.sort(reverse=False)
            saveSortedListAsTextFile(parsedWords, cipher_shorts[i] + "_132_ninja", key1)


KEY 1: A
KEY 1: B
KEY 1: C
KEY 1: D
KEY 1: E
KEY 1: F
KEY 1: G
KEY 1: H
KEY 1: I
KEY 1: J
KEY 1: K
KEY 1: L
KEY 1: M
KEY 1: N
KEY 1: O
KEY 1: P
KEY 1: Q
KEY 1: R
KEY 1: S
KEY 1: T
KEY 1: U
KEY 1: V
KEY 1: W
KEY 1: X
KEY 1: Y
KEY 1: Z


In [17]:
# notice that the maximum length of short ciphers is 25, less than 1 compared to the alphabets A to Z
cipher_shorts = [
    #"FEWGDRHDDEEUMFFTEEMJXZR",
    #"UGMNCBXCFLDBEY",
    #"GKJFHYXODIE",
    
    #"ABRYCTUGVZXUPB",
    #"HFXPCQYZVATXAWIZPVE",
    #"HLMTAHGBGFNIV",
    #"JKGFIJPMCWSAEK",
    #"KOWVRSRKWTMLDH",
    #"MQOLCSJTLGAJOKBSSBOMUPCE",
    "MVERZRLQDBHQ",
    #"RHZVIYQIYSXVNQXQWIOVWPJO",
    #"SKCDKJCDJCYQSZKTZJPXPWIRN",
    #"VIOHIKNNGUAB",
    #"YQHUDTABGALLOWLS",
    #"ZUQUPNZN"
]

cipher_shorts_rev = []
for i in range(len(cipher_shorts)):
    cipher_shorts_rev.append(cipher_shorts[i][::-1])
#print(cipher_shorts_rev)

# start with 'H', do 'G' at the end since it rotates the second rotor
alphabets = "ABCDEFGHIJKLMNOPQRSTUVWXYZ"
alpha1 = alphabets
alpha2 = alphabets
alpha3 = alphabets
keys2 = []
for x in alpha2:
    for y in alpha3:
        keys2.append(x + y)

for k in range(len(alpha1)):
    key1 = alpha1[k]
    print("KEY 1: " + key1)
    for i in range(len(cipher_shorts)):
        parsedWords = []
        decList1 = []
        decList2 = []
        for j in range(len(keys2)):
            keyset = key1 + keys2[j]
            #
            engine = Enigma(ROTOR_UKW_K, ROTOR_I_K, ROTOR_II_K, ROTOR_III_K, key=keyset, ring='AAA', debug=False)
            decoded = engine.encipher(cipher_shorts_rev[i]).upper()
            parsed = ' '.join(wordninja.split(decoded))
            parsedWords.append(parsed + " : " + decoded + ": " + keyset)
        #
        if len(parsedWords):
            parsedWords.sort(reverse=False)
            saveSortedListAsTextFile(parsedWords, cipher_shorts[i] + "_123_ninja_rev", key1)

KEY 1: A
KEY 1: B
KEY 1: C
KEY 1: D
KEY 1: E
KEY 1: F
KEY 1: G
KEY 1: H
KEY 1: I
KEY 1: J
KEY 1: K
KEY 1: L
KEY 1: M
KEY 1: N
KEY 1: O
KEY 1: P
KEY 1: Q
KEY 1: R
KEY 1: S
KEY 1: T
KEY 1: U
KEY 1: V
KEY 1: W
KEY 1: X
KEY 1: Y
KEY 1: Z


In [6]:
# notice that the maximum length of short ciphers is 25, less than 1 compared to the alphabets A to Z
cipher_shorts = [
    #"FEWGDRHDDEEUMFFTEEMJXZR",
    #"UGMNCBXCFLDBEY",
    #"GKJFHYXODIE",
    
    #"ABRYCTUGVZXUPB",
    #"HFXPCQYZVATXAWIZPVE",
    #"HLMTAHGBGFNIV",
    #"JKGFIJPMCWSAEK",
    #"KOWVRSRKWTMLDH",
    #"MQOLCSJTLGAJOKBSSBOMUPCE",
    "MVERZRLQDBHQ",
    #"RHZVIYQIYSXVNQXQWIOVWPJO",
    #"SKCDKJCDJCYQSZKTZJPXPWIRN",
    #"VIOHIKNNGUAB",
    #"YQHUDTABGALLOWLS",
    #"ZUQUPNZN"
]

# start with 'H', do 'G' at the end since it rotates the second rotor
alphabets = "ABCDEFGHIJKLMNOPQRSTUVWXYZ"
alpha1 = alphabets
alpha2 = alphabets
alpha3 = alphabets
keys2 = []
for x in alpha2:
    for y in alpha3:
        keys2.append(x + y)

for k in range(len(alpha1)):
    key1 = alpha1[k]
    print("KEY 1: " + key1)
    for i in range(len(cipher_shorts)):
        decList1 = []
        decList2 = []
        for j in range(len(keys2)):
            keyset = key1 + keys2[j]
            engine = Enigma(ROTOR_UKW_K, ROTOR_I_K, ROTOR_II_K, ROTOR_III_K, key=keyset, ring='AAA', debug=False)
            decoded = engine.encipher(cipher_shorts[i]).upper()
            if isFreeConsonantMoreThan(decoded, 3):
                decList2.append(decoded + ", keys: " + keyset + " : " + cipher_shorts[i])
            else:
                decList1.append(decoded + ", keys: " + keyset + " : " + cipher_shorts[i])
        if len(decList1) > 0:
            decList1.sort()
            saveSortedListAsTextFile(decList1, cipher_shorts[i] + "_123", key1)
        if len(decList2) > 0:
            decList2.sort()
            saveSortedListAsTextFile(decList2, cipher_shorts[i] + "_123", key1 + "_X")

['QHBDQLRZREVM']
KEY 1: A
KEY 1: B
KEY 1: C
KEY 1: D
KEY 1: E
KEY 1: F
KEY 1: G
KEY 1: H
KEY 1: I
KEY 1: J
KEY 1: K
KEY 1: L
KEY 1: M
KEY 1: N
KEY 1: O
KEY 1: P
KEY 1: Q
KEY 1: R
KEY 1: S
KEY 1: T
KEY 1: U
KEY 1: V
KEY 1: W
KEY 1: X
KEY 1: Y
KEY 1: Z


In [19]:
k = "JQE"
r = "AAA"
secret = "FEWGDRHDDEEUMFFTEEMJXZR"
print('key = ', k, ', ring: ', r, '\n', 'secret: ', secret)

engine = Enigma(ROTOR_UKW_K, ROTOR_I_K, ROTOR_II_K, ROTOR_III_K, key=k, ring=r, debug=True)
text = engine.encipher(secret[0:23:1]).upper()

print(text)

key =  JQE , ring:  AAA 
 secret:  FEWGDRHDDEEUMFFTEEMJXZR
I-K [>>>] J: F I
II-K [>>>] Q: I N
III-K [>>>] E: N U
UKW-K [---] A: U V
III-K [<<<] E: V J
II-K [<<<] Q: J A
I-K [<<<] J: A O
I-K [turnover] ? = False, state = K, notchs = G
I-K [notch] J --> K
[states] : KQE
I-K [>>>] K: E H
II-K [>>>] Q: H X
III-K [>>>] E: X F
UKW-K [---] A: F G
III-K [<<<] E: G D
II-K [<<<] Q: D P
I-K [<<<] K: P U
I-K [turnover] ? = False, state = L, notchs = G
I-K [notch] K --> L
[states] : LQE
I-K [>>>] L: W P
II-K [>>>] Q: P D
III-K [>>>] E: D G
UKW-K [---] A: G F
III-K [<<<] E: F X
II-K [<<<] Q: X H
I-K [<<<] L: H S
I-K [turnover] ? = False, state = M, notchs = G
I-K [notch] L --> M
[states] : MQE
I-K [>>>] M: G I
II-K [>>>] Q: I N
III-K [>>>] E: N U
UKW-K [---] A: U V
III-K [<<<] E: V J
II-K [<<<] Q: J A
I-K [<<<] M: A T
I-K [turnover] ? = False, state = N, notchs = G
I-K [notch] M --> N
[states] : NQE
I-K [>>>] N: D L
II-K [>>>] Q: L V
III-K [>>>] E: V A
UKW-K [---] A: A I
III-K [<<<] E: I M
II-K [<<<

In [25]:
plugs = "YQ HU DT AB GA LL OW LS"
print(plugs.split())

p = [(elem[0], elem[1]) for elem in plugs.split()]
print(p)

['YQ', 'HU', 'DT', 'AB', 'GA', 'LL', 'OW', 'LS']
[('Y', 'Q'), ('H', 'U'), ('D', 'T'), ('A', 'B'), ('G', 'A'), ('L', 'L'), ('O', 'W'), ('L', 'S')]
